<a href="https://colab.research.google.com/github/EMORI-SERVULO/Public/blob/master/RED-NEURONAL-EFICIENCIA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np


In [0]:
dataset = np.loadtxt("https://smarteyes.ga/TABLA-TEST-EFICIENCIA.csv", delimiter=";")

In [10]:
#print(X)
#print(Y)

[79.76 79.68 79.73 ... 83.45 82.79 82.65]


In [0]:
def sigmoid(x):
    return 1.0/(1.0 + np.exp(-x))

def sigmoid_derivada(x):
    return sigmoid(x)*(1.0-sigmoid(x))

def tanh(x):
    return np.tanh(x)

def tanh_derivada(x):
    return 1.0 - x**2

In [0]:
class NeuralNetwork:

    def __init__(self, layers, activation='tanh'):
        if activation == 'sigmoid':
            self.activation = sigmoid
            self.activation_prime = sigmoid_derivada
        elif activation == 'tanh':
            self.activation = tanh
            self.activation_prime = tanh_derivada

        # inicializo los pesos
        self.weights = []
        self.deltas = []
        # capas = [2,3,2]
        # rando de pesos varia entre (-1,1)
        # asigno valores aleatorios a capa de entrada y capa oculta
        for i in range(1, len(layers) - 1):
            r = 2*np.random.random((layers[i-1] + 1, layers[i] + 1)) -1
            self.weights.append(r)
        # asigno aleatorios a capa de salida
        r = 2*np.random.random( (layers[i] + 1, layers[i+1])) - 1
        self.weights.append(r)

    def fit(self, X, y, learning_rate=0.2, epochs=100000):
        # Agrego columna de unos a las entradas X
        # Con esto agregamos la unidad de Bias a la capa de entrada
        ones = np.atleast_2d(np.ones(X.shape[0]))
        X = np.concatenate((ones.T, X), axis=1)
        
        for k in range(epochs):
            i = np.random.randint(X.shape[0])
            a = [X[i]]

            for l in range(len(self.weights)):
                    dot_value = np.dot(a[l], self.weights[l])
                    activation = self.activation(dot_value)
                    a.append(activation)
            # Calculo la diferencia en la capa de salida y el valor obtenido
            error = y[i] - a[-1]
            deltas = [error * self.activation_prime(a[-1])]
            
            # Empezamos en el segundo layer hasta el ultimo
            # (Una capa anterior a la de salida)
            for l in range(len(a) - 2, 0, -1): 
                deltas.append(deltas[-1].dot(self.weights[l].T)*self.activation_prime(a[l]))
            self.deltas.append(deltas)

            # invertir
            # [level3(output)->level2(hidden)]  => [level2(hidden)->level3(output)]
            deltas.reverse()

            # backpropagation
            # 1. Multiplcar los delta de salida con las activaciones de entrada 
            #    para obtener el gradiente del peso.
            # 2. actualizo el peso restandole un porcentaje del gradiente
            for i in range(len(self.weights)):
                layer = np.atleast_2d(a[i])
                delta = np.atleast_2d(deltas[i])
                self.weights[i] += learning_rate * layer.T.dot(delta)

            if k % 10000 == 0: print('epochs:', k)

    def predict(self, x): 
        ones = np.atleast_2d(np.ones(x.shape[0]))
        a = np.concatenate((np.ones(1).T, np.array(x)), axis=0)
        for l in range(0, len(self.weights)):
            a = self.activation(np.dot(a, self.weights[l]))
        return a

    def print_weights(self):
        print("LISTADO PESOS DE CONEXIONES")
        for i in range(len(self.weights)):
            print(self.weights[i])

    def get_deltas(self):
        return self.deltas

In [22]:
nn = NeuralNetwork([6,5,1],activation ='tanh')

X = dataset[:,0:6]
y = dataset[:,6]
nn.fit(X, y,epochs=1000)
for e in X:
    print("Entrdas:",e,"Salidas:",nn.predict(e))

epochs: 0
Entrdas: [193.65  44.69 104.36 174.65   8.37   3.59] Salidas: [0.99999982]
Entrdas: [192.07  45.98 103.7  174.65   8.39   3.65] Salidas: [0.99999982]
Entrdas: [190.09  45.91 104.89 174.72   8.4    4.  ] Salidas: [0.99999982]
Entrdas: [193.82  45.28 105.61 174.52   8.35   4.38] Salidas: [0.99999982]
Entrdas: [194.35  45.15 105.98 174.65   8.39   3.53] Salidas: [0.99999982]
Entrdas: [195.01  45.42 104.23 174.65   8.38   3.42] Salidas: [0.99999982]
Entrdas: [187.22  45.18 103.07 174.72   8.43   4.1 ] Salidas: [0.99999982]
Entrdas: [187.75  44.72 103.73 174.62   8.38   4.55] Salidas: [0.99999982]
Entrdas: [189.93  44.76 103.87 174.56   8.39   4.17] Salidas: [0.99999982]
Entrdas: [188.34  44.99 102.91 174.62   8.37   4.25] Salidas: [0.99999982]
Entrdas: [188.81  44.62 105.22 174.56   8.37   4.23] Salidas: [0.99999982]
Entrdas: [190.22  44.39 106.18 174.59   8.39   4.27] Salidas: [0.99999982]
Entrdas: [193.82  44.76 107.53 174.56   8.34   3.72] Salidas: [0.99999982]
Entrdas: [192.8

In [23]:
import matplotlib.pyplot as plt

deltas = nn.get_deltas()
valores=[]
index=0
for arreglo in deltas:
    valores.append(arreglo[1][0] + arreglo[1][1])
    index=index+1

plt.plot(range(len(valores)), valores, color='b')
plt.ylim([0, 1])
plt.ylabel('Cost')
plt.xlabel('Epochs')
plt.tight_layout()
plt.show()

IndexError: ignored